In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind_log"
label_col = "label"

# abnormal return 컬럼
abn_return_cols = [
    "abn_ret_minus_9m", "abn_ret_minus_8m", "abn_ret_minus_7m", "abn_ret_minus_6m",
    "abn_ret_minus_5m", "abn_ret_minus_4m", "abn_ret_minus_3m", "abn_ret_minus_2m", "abn_ret_minus_1m", "abn_ret_0m",
    "abn_ret_1m", "abn_ret_2m", "abn_ret_3m", "abn_ret_4m", "abn_ret_5m", "abn_ret_6m",
    "abn_ret_7m", "abn_ret_8m", "abn_ret_9m", "abn_ret_10m"
]

# disclosure_type도 추가
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"] +
    [f"k.disclosure_type"]
)
sql_columns_str = ",\n    ".join(sql_columns)

# 2021-01-01 ~ 2022-06-30(포함) event_ts 조건 추가
query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
WHERE ar.event_ts >= '2021-01-01' AND ar.event_ts <= '2022-06-30'
ORDER BY ar.event_ts ASC
"""

df_before_telegram = pd.read_sql(query, engine)
df_before_telegram["period_dummy"] = 0
df_before_telegram

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,abn_ret_minus_1m,...,abn_ret_4m,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,disclosure_type,period_dummy
0,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,0.003677,-0.003850,-0.000371,-0.000044,-0.004355,0.003281,-0.001078,0,단일판매ㆍ공급계약체결,0
1,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,0.003677,-0.003850,-0.000371,-0.000044,-0.004355,0.003281,-0.001078,0,단일판매ㆍ공급계약체결,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,-0.001945,0.001952,-0.008174,-0.002652,-0.001960,0.007570,-0.005890,...,0.001634,0.002292,0.010508,-0.003930,0.002619,0.003281,-0.008866,1,단일판매ㆍ공급계약체결,0
3,2021-01-04 10:14:00+00:00,-0.001960,0.003144,0.000105,0.001302,-0.004985,0.002281,0.000653,0.002719,0.003376,...,-0.014566,0.003788,0.001318,-0.001263,-0.005605,-0.005578,-0.001972,1,품목허가 승인,0
4,2021-01-04 10:16:00+00:00,0.002785,-0.002659,-0.000252,0.000252,0.000000,-0.000775,-0.001412,-0.003190,0.001265,...,-0.000634,-0.001266,-0.003063,0.001799,-0.001768,-0.002558,0.005234,-1,단일판매ㆍ공급계약체결,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,2022-06-29 14:04:00+00:00,-0.004574,-0.000476,0.004732,-0.000159,-0.021638,0.021796,-0.003939,-0.000317,-0.026907,...,-0.021662,0.009167,-0.008691,0.000793,0.000318,-0.000476,-0.008452,1,품목허가 승인,0
5068,2022-06-29 14:20:00+00:00,0.000793,0.000318,-0.000476,0.000476,0.000159,-0.000635,-0.000635,-0.000159,0.000159,...,-0.001927,0.000000,0.001927,-0.001292,0.000635,0.000318,-0.000318,1,매출액변동,0
5069,2022-06-29 14:21:00+00:00,0.001530,-0.001689,-0.000738,0.000159,0.000579,-0.000635,0.001054,-0.001054,-0.002589,...,-0.001213,0.000159,0.001689,0.001847,-0.000894,-0.000318,-0.001054,1,단일판매ㆍ공급계약체결,0
5070,2022-06-29 14:35:00+00:00,0.000159,-0.000960,0.000635,-0.002560,0.001122,0.000159,-0.002882,0.002247,-0.001281,...,0.001912,0.000159,-0.002715,-0.000318,-0.000318,-0.000159,-0.001599,1,단일판매ㆍ공급계약체결,0


In [3]:
label_table = "label"
return_table = "abnormal_return_log"
label_col = "label"


all_sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"] +
    ["k.report_name"] +
    ["k.summary_kr"]
)
all_sql_columns_str = ",\n    ".join(all_sql_columns)

query = f"""
SELECT {all_sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
JOIN disclosure_events de ON ar.event_id = de.id
WHERE k.{label_col} IS NOT NULL
ORDER BY ar.event_ts ASC
"""

df_after_telegram = pd.read_sql(query, engine)
df_after_telegram["period_dummy"] = 1
label_map = {0: 1, 1: 0, 2: -1}
df_after_telegram['label'] = df_after_telegram['label'].map(label_map)
df_after_telegram

,event_ts,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,abn_ret_minus_1m,...,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,report_name,summary_kr,period_dummy
0,2022-07-01 01:03:17+00:00,0.011478,0.008055,0.000474,-0.003784,-0.000001,0.004726,-0.004252,0.000000,0.001898,...,-0.000946,-0.002850,-0.003819,0.003342,-0.004794,-0.004810,1,타법인주식및출자증권취득결정(자율공시),"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...",1
1,2022-07-01 01:24:45+00:00,0.001432,0.000000,0.000000,0.001434,0.000479,0.000479,-0.004817,0.000000,-0.001435,...,0.001920,-0.003784,-0.003802,0.003802,-0.003802,0.004283,1,단일판매ㆍ공급계약체결,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,1
2,2022-07-01 01:27:02+00:00,0.001434,-0.000959,0.000479,-0.001919,0.000000,0.000005,0.000956,0.001916,-0.001437,...,-0.001927,-0.000967,0.000000,0.000967,0.002889,0.001445,1,단일판매ㆍ공급계약체결(자회사의 주요경영사항),비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,1
3,2022-07-01 01:43:15+00:00,-0.001049,-0.001140,-0.000238,0.000979,-0.000979,-0.000489,-0.000815,-0.000814,0.000000,...,-0.003846,0.009345,-0.006474,0.006515,-0.007813,0.001136,1,신규시설투자등(자율공시),동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,1
4,2022-07-01 02:30:09+00:00,-0.003299,0.004264,-0.002817,0.001851,0.000483,-0.000483,-0.000483,0.001846,0.002323,...,0.003279,0.004125,-0.002785,0.004598,-0.001813,0.000000,-1,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,2023-12-28 05:53:08+00:00,0.007121,0.000417,-0.000555,-0.006324,0.000278,-0.000139,0.000000,-0.000139,-0.000416,...,-0.000743,0.000000,0.000139,0.000416,0.000000,-0.000277,1,단일판매ㆍ공급계약체결,까뮤이앤씨는 조달청과 전라남도 목포시에서 수산식품 수출단지 조성사업 건축공사(공급지...,1
8759,2023-12-28 05:55:03+00:00,0.000499,0.002355,-0.000774,-0.002774,-0.001584,0.001973,-0.000944,-0.001221,0.002636,...,-0.010608,0.006878,0.003215,0.007716,-0.003722,-0.006691,0,조회공시요구(현저한시황변동)에대한답변(미확정),"에코프로머티리얼즈는 신규 고객 확보와 관련된 사업 협의를 진행 중이며, 아직 확정된...",1
8760,2023-12-28 05:56:27+00:00,-0.001326,0.001931,0.000604,-0.001684,-0.001684,0.001324,-0.002290,0.000719,-0.000360,...,0.002656,-0.001936,0.002296,-0.001936,-0.000249,0.001578,1,[기재정정]연결재무제표기준영업실적등에대한전망(공정공시),"에코마케팅은 2023년 매출액 3,500억 원과 영업이익 530억 원을 기록할 것으...",1
8761,2023-12-28 05:56:32+00:00,-0.001326,0.001931,0.000604,-0.001684,-0.001684,0.001324,-0.002290,0.000719,-0.000360,...,0.002656,-0.001936,0.002296,-0.001936,-0.000249,0.001578,1,[기재정정]영업실적등에대한전망(공정공시),"에코마케팅은 2023년 매출액이 490억 원, 영업이익이 200억 원으로 예상되며,...",1


In [4]:
from config import keywords
import re
import os

def classify_report_name_with_keywords_return_series_and_list(df, keywords):
    """
    df: DataFrame, must include 'report_name' column
    keywords: dict, key=disclosure_type, value=[regex strings]
    
    Returns: (Series of classified disclosure_type, List of disclosure_types in order of df)
    """
    pattern_list = []
    for disclosure_type, pattern_strs in keywords.items():
        if pattern_strs:
            for pat in pattern_strs:
                pattern_list.append( (disclosure_type, re.compile(pat)) )
        else:
            pattern_list.append( (disclosure_type, re.compile(disclosure_type)) )

    def classify_single_report(report_name):
        for disclosure_type, pat in pattern_list:
            if isinstance(report_name, str):
                if pat.search(report_name):
                    return disclosure_type
        return None

    result_list = df["report_name"].apply(classify_single_report).tolist()
    return pd.Series(result_list, index=df.index), result_list

output_dir = "type"
os.makedirs(output_dir, exist_ok=True)

# 분류도 csv로 저장하고 df_after_telegram['disclosure_type']에도 넣기
disclosure_type_series, disclosure_type_list = classify_report_name_with_keywords_return_series_and_list(
    df_after_telegram, keywords)
df_after_telegram["disclosure_type"] = disclosure_type_series

# 각 event type(disclosure_type) 별로 따로 csv 저장 (type 폴더에!)
for dtype, group in df_after_telegram.groupby("disclosure_type"):
    if dtype is None:
        continue  # 기타 처리로 분리
    fname = f"{dtype}.csv".replace("/", "_").replace("\\", "_")
    path = os.path.join(output_dir, fname)
    # summary_kr도 같이 저장 (없으면 없는 컬럼으로 저장)
    cols_to_save = ["report_name"]
    if "summary_kr" in group.columns:
        cols_to_save.append("summary_kr")
    group[cols_to_save].to_csv(path, index=False)

# 분류되지 않은 event는 "기타.csv"로 저장
not_classified = df_after_telegram[df_after_telegram['disclosure_type'].isna()]
if not not_classified.empty:
    etc_path = os.path.join(output_dir, "기타.csv")
    cols_to_save = ["report_name"]
    if "summary_kr" in not_classified.columns:
        cols_to_save.append("summary_kr")
    not_classified[cols_to_save].to_csv(etc_path, index=False)


In [5]:
from config import keywords

# 제외할 disclosure_type 목록
exclude_types = ["실적공시", "지분공시", "IR활동", "배당"]

# disclosure_type이 keywords key에 있으면서 exclude_types에는 없는 값만 보여주기
filtered_types = [k for k in keywords.keys() if k not in exclude_types]
df_after_telegram[df_after_telegram["disclosure_type"].isin(filtered_types)]["disclosure_type"].value_counts()

disclosure_type
단일판매ㆍ공급계약체결           1054
매출액변동                  351
자기주식취득 신탁계약 체결 결정      205
타법인 주식 및 출자증권 양수결정     110
자기주식 취득 결정              75
특허권취득                   74
자기주식 처분 결정              68
신규시설투자                  67
유상증자 결정                 62
임상 계획 신청                51
무상증자 결정                 49
소송등의판결ㆍ결정               28
임상 계획 승인                27
전환사채권 발행결정              22
자기주식 소각 결정              21
타법인 주식 및 출자증권 양도결정      20
소송 등의 제기                19
유형자산 양수 결정              11
회사분할 결정                  9
회사합병 결정                  9
품목허가 신청                  9
생산중단                     7
임상 계획 결과 발표              7
감자 결정                    4
자기주식취득 신탁계약 해지 결정        3
품목허가 승인                  3
임상 계획 철회                 3
기술이전계약체결                 3
영업양도 결정                  2
영업양수 결정                  2
품목허가 철회                  2
주식교환ㆍ이전 결정               1
Name: count, dtype: int64

In [6]:
df_after_telegram_filtered = df_after_telegram[df_after_telegram["disclosure_type"].isin(filtered_types)]
df_after_telegram_filtered

,event_ts,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,abn_ret_minus_1m,...,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,report_name,summary_kr,period_dummy,disclosure_type
0,2022-07-01 01:03:17+00:00,0.011478,0.008055,0.000474,-0.003784,-0.000001,0.004726,-0.004252,0.000000,0.001898,...,-0.002850,-0.003819,0.003342,-0.004794,-0.004810,1,타법인주식및출자증권취득결정(자율공시),"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...",1,타법인 주식 및 출자증권 양수결정
1,2022-07-01 01:24:45+00:00,0.001432,0.000000,0.000000,0.001434,0.000479,0.000479,-0.004817,0.000000,-0.001435,...,-0.003784,-0.003802,0.003802,-0.003802,0.004283,1,단일판매ㆍ공급계약체결,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,1,단일판매ㆍ공급계약체결
2,2022-07-01 01:27:02+00:00,0.001434,-0.000959,0.000479,-0.001919,0.000000,0.000005,0.000956,0.001916,-0.001437,...,-0.000967,0.000000,0.000967,0.002889,0.001445,1,단일판매ㆍ공급계약체결(자회사의 주요경영사항),비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,1,단일판매ㆍ공급계약체결
3,2022-07-01 01:43:15+00:00,-0.001049,-0.001140,-0.000238,0.000979,-0.000979,-0.000489,-0.000815,-0.000814,0.000000,...,0.009345,-0.006474,0.006515,-0.007813,0.001136,1,신규시설투자등(자율공시),동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,1,신규시설투자
4,2022-07-01 02:30:09+00:00,-0.003299,0.004264,-0.002817,0.001851,0.000483,-0.000483,-0.000483,0.001846,0.002323,...,0.004125,-0.002785,0.004598,-0.001813,0.000000,-1,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,1,임상 계획 신청
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2023-12-28 04:10:54+00:00,0.000418,-0.000139,0.000279,0.005546,-0.001613,0.000139,0.000139,-0.000585,0.000139,...,-0.011871,-0.009388,0.006507,-0.002332,0.009929,1,단일판매ㆍ공급계약체결,에쓰씨엔지니어링은 2023년 12월 28일 전북 군산시에서 NEXEON Korea와...,1,단일판매ㆍ공급계약체결
8754,2023-12-28 04:54:23+00:00,0.000000,-0.000361,-0.000361,0.000000,0.003656,-0.002187,0.004010,-0.000721,0.000361,...,0.000000,-0.000360,0.000000,0.000000,-0.002180,1,기타경영사항(자율공시)(펩타이드함유 더말 필러(레보필울트라)의 주름개선 효과와 안전...,케어젠은 펩타이드 함유 더말 필러(레보필 울트라)의 주름개선 효과와 안전성을 확인하...,1,임상 계획 승인
8756,2023-12-28 05:42:24+00:00,-0.000139,-0.000417,-0.000417,-0.000278,0.000000,-0.000278,0.000000,0.000000,-0.000139,...,0.000278,-0.000139,0.003249,-0.000139,0.000664,1,단일판매ㆍ공급계약체결,한창제지는 케이티앤지와 내자 아이보리판지 구매를 위한 455억 원 규모의 판매·공급...,1,단일판매ㆍ공급계약체결
8757,2023-12-28 05:47:03+00:00,0.000944,0.000000,0.000000,0.000360,0.000000,-0.001304,-0.000360,0.000000,0.019996,...,0.003071,-0.006153,0.004799,-0.003938,-0.001941,1,단일판매ㆍ공급계약체결,윤성에프앤씨는 2023년 12월 27일부터 2024년 11월 16일까지 10개월간 ...,1,단일판매ㆍ공급계약체결


In [7]:
df_total = pd.concat([df_before_telegram, df_after_telegram_filtered])



In [8]:
df_total

,event_ts,abn_ret_minus_9m,abn_ret_minus_8m,abn_ret_minus_7m,abn_ret_minus_6m,abn_ret_minus_5m,abn_ret_minus_4m,abn_ret_minus_3m,abn_ret_minus_2m,abn_ret_minus_1m,...,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,abn_ret_10m,label,disclosure_type,period_dummy,report_name,summary_kr
0,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,-0.000371,-0.000044,-0.004355,0.003281,-0.001078,0,단일판매ㆍ공급계약체결,0,NaN,NaN
1,2021-01-04 10:08:00+00:00,0.000000,0.009472,-0.001189,-0.006321,-0.004620,0.003331,-0.004050,0.007088,-0.002013,...,-0.000371,-0.000044,-0.004355,0.003281,-0.001078,0,단일판매ㆍ공급계약체결,0,NaN,NaN
2,2021-01-04 10:08:00+00:00,NaN,NaN,-0.001945,0.001952,-0.008174,-0.002652,-0.001960,0.007570,-0.005890,...,0.010508,-0.003930,0.002619,0.003281,-0.008866,1,단일판매ㆍ공급계약체결,0,NaN,NaN
3,2021-01-04 10:14:00+00:00,-0.001960,0.003144,0.000105,0.001302,-0.004985,0.002281,0.000653,0.002719,0.003376,...,0.001318,-0.001263,-0.005605,-0.005578,-0.001972,1,품목허가 승인,0,NaN,NaN
4,2021-01-04 10:16:00+00:00,0.002785,-0.002659,-0.000252,0.000252,0.000000,-0.000775,-0.001412,-0.003190,0.001265,...,-0.003063,0.001799,-0.001768,-0.002558,0.005234,-1,단일판매ㆍ공급계약체결,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8751,2023-12-28 04:10:54+00:00,0.000418,-0.000139,0.000279,0.005546,-0.001613,0.000139,0.000139,-0.000585,0.000139,...,-0.011871,-0.009388,0.006507,-0.002332,0.009929,1,단일판매ㆍ공급계약체결,1,단일판매ㆍ공급계약체결,에쓰씨엔지니어링은 2023년 12월 28일 전북 군산시에서 NEXEON Korea와...
8754,2023-12-28 04:54:23+00:00,0.000000,-0.000361,-0.000361,0.000000,0.003656,-0.002187,0.004010,-0.000721,0.000361,...,0.000000,-0.000360,0.000000,0.000000,-0.002180,1,임상 계획 승인,1,기타경영사항(자율공시)(펩타이드함유 더말 필러(레보필울트라)의 주름개선 효과와 안전...,케어젠은 펩타이드 함유 더말 필러(레보필 울트라)의 주름개선 효과와 안전성을 확인하...
8756,2023-12-28 05:42:24+00:00,-0.000139,-0.000417,-0.000417,-0.000278,0.000000,-0.000278,0.000000,0.000000,-0.000139,...,0.000278,-0.000139,0.003249,-0.000139,0.000664,1,단일판매ㆍ공급계약체결,1,단일판매ㆍ공급계약체결,한창제지는 케이티앤지와 내자 아이보리판지 구매를 위한 455억 원 규모의 판매·공급...
8757,2023-12-28 05:47:03+00:00,0.000944,0.000000,0.000000,0.000360,0.000000,-0.001304,-0.000360,0.000000,0.019996,...,0.003071,-0.006153,0.004799,-0.003938,-0.001941,1,단일판매ㆍ공급계약체결,1,단일판매ㆍ공급계약체결,윤성에프앤씨는 2023년 12월 27일부터 2024년 11월 16일까지 10개월간 ...


In [9]:
# config.py의 keywords 딕셔너리를 사용해서 한글-영어 카테고리 매핑 생성
category_mapping = {
    # 임상 관련
    "임상 계획 철회": "Clinical Trial Withdrawal",
    "임상 계획 신청": "Clinical Trial Application", 
    "임상 계획 승인": "Clinical Trial Approval",
    "임상 계획 결과 발표": "Clinical Trial Results",
    
    # 자산 관련
    "자산양수도(기타), 풋백옵션": "Asset Transfer & Putback Option",
    
    # 경영 관련
    "부도발생": "Default Occurrence",
    "영업정지": "Business Suspension", 
    "회생절차 개시신청": "Rehabilitation Procedure Initiation",
    "해산사유 발생": "Dissolution Cause Occurrence",
    
    # 자본 변동
    "유상증자 결정": "Paid-in Capital Increase Decision",
    "무상증자 결정": "Free Capital Increase Decision", 
    "유무상증자 결정": "Paid/Free Capital Increase Decision",
    "감자 결정": "Capital Reduction Decision",
    "주식분할 결정": "Stock Split Decision",
    
    # 금융 관련
    "채권은행 등의 관리절차 개시": "Creditor Bank Management Procedure Initiation",
    "채권은행 등의 관리절차 중단": "Creditor Bank Management Procedure Suspension",
    
    # 소송/분쟁
    "소송 등의 제기": "Litigation Filing",
    "소송등의판결ㆍ결정": "Litigation Judgment/Decision",
    
    # 해외 상장
    "해외 증권시장 주권등 상장 결정": "Overseas Securities Market Listing Decision",
    "해외 증권시장 주권등 상장폐지 결정": "Overseas Securities Market Delisting Decision", 
    "해외 증권시장 주권등 상장": "Overseas Securities Market Listing",
    "해외 증권시장 주권등 상장폐지": "Overseas Securities Market Delisting",
    
    # 채권 발행
    "전환사채권 발행결정": "Convertible Bond Issuance Decision",
    "신주인수권부사채권 발행결정": "Bond with Warrant Issuance Decision",
    "교환사채권 발행결정": "Exchangeable Bond Issuance Decision",
    "상각형 조건부자본증권 발행결정": "Contingent Convertible Security Issuance Decision",
    
    # 자사주 관련
    "자기주식 취득 결정": "Treasury Stock Acquisition Decision",
    "자기주식 처분 결정": "Treasury Stock Disposal Decision", 
    "자기주식 소각 결정": "Treasury Stock Retirement Decision",
    "자기주식취득 신탁계약 체결 결정": "Treasury Stock Acquisition Trust Contract Decision",
    "자기주식취득 신탁계약 해지 결정": "Treasury Stock Acquisition Trust Contract Termination Decision",
    
    # 영업 양수도
    "영업양수 결정": "Business Acquisition Decision",
    "영업양도 결정": "Business Transfer Decision",
    
    # 자산 취득/처분
    "유형자산 양수 결정": "Tangible Asset Acquisition Decision",
    "유형자산 양도 결정": "Tangible Asset Transfer Decision",
    "타법인 주식 및 출자증권 양수결정": "Other Corporation Stock Acquisition Decision",
    "타법인 주식 및 출자증권 양도결정": "Other Corporation Stock Transfer Decision",
    "주권 관련 사채권 양수 결정": "Stock-related Bond Acquisition Decision",
    "주권 관련 사채권 양도 결정": "Stock-related Bond Transfer Decision",
    
    # 기업 구조 변경
    "회사합병 결정": "Company Merger Decision",
    "회사분할 결정": "Company Spin-off Decision", 
    "회사분할합병 결정": "Company Split-Merger Decision",
    "주식교환ㆍ이전 결정": "Stock Exchange/Transfer Decision",
    
    # 공시 관련
    "지분공시": "Equity Disclosure",
    "실적공시": "Earnings Disclosure",
    
    # 계약 관련
    "단일판매ㆍ공급계약해지": "Single Sales/Supply Contract Termination",
    "단일판매ㆍ공급계약체결": "Single Sales/Supply Contract Execution",
    
    # 기타
    "생산중단": "Production Suspension",
    "배당": "Dividend",
    "매출액변동": "Revenue Change",
    "특허권취득": "Patent Acquisition",
    "신규시설투자": "New Facility Investment",
    "기술이전계약해지": "Technology Transfer Contract Termination",
    "기술이전계약체결": "Technology Transfer Contract Execution",
    
    # 품목허가 관련
    "품목허가 철회": "Product Approval Withdrawal",
    "품목허가 신청": "Product Approval Application",
    "품목허가 승인": "Product Approval",
    
    # 기타
    "횡령ㆍ배임혐의발생": "Embezzlement/Malfeasance Allegation",
    "공개매수": "Public Tender Offer"
}

import pandas as pd
pd.DataFrame(list(category_mapping.items()), columns=["category_kr", "category_en"])



# period_dummy(0,1)에 따라 disclosure_type별 건수 계산
counts_by_period = (
    df_total.groupby(['disclosure_type', 'period_dummy'])
    .size()
    .unstack(fill_value=0)
    .rename(columns={0: "observations_before", 1: "observations_after"})
)

# 카테고리 기준으로 정리 (category_mapping 기준, 없는 것은 0으로)
category_df = pd.DataFrame([
    {
        "category_kr": cat,
        "category_en": category_mapping[cat],
        "observations_before": counts_by_period.loc[cat, "observations_before"] if cat in counts_by_period.index else 0,
        "observations_after": counts_by_period.loc[cat, "observations_after"] if cat in counts_by_period.index else 0,
    }
    for cat in category_mapping.keys()
    if cat in df_total['disclosure_type'].unique()
])


category_df["observations_total"] = category_df["observations_before"] + category_df["observations_after"]
category_df = category_df.sort_values("observations_total", ascending=False)

# 인덱스를 1, 2, ...로 순서대로 지정
category_df.index = range(1, len(category_df) + 1)

display(category_df)

,category_kr,category_en,observations_before,observations_after,observations_total
1,단일판매ㆍ공급계약체결,Single Sales/Supply Contract Execution,1838,1054,2892
2,매출액변동,Revenue Change,1354,351,1705
3,특허권취득,Patent Acquisition,378,74,452
4,자기주식취득 신탁계약 체결 결정,Treasury Stock Acquisition Trust Contract Deci...,192,205,397
5,타법인 주식 및 출자증권 양수결정,Other Corporation Stock Acquisition Decision,278,110,388
6,자기주식 취득 결정,Treasury Stock Acquisition Decision,108,75,183
7,자기주식 처분 결정,Treasury Stock Disposal Decision,112,68,180
8,신규시설투자,New Facility Investment,106,67,173
9,무상증자 결정,Free Capital Increase Decision,101,49,150
10,유상증자 결정,Paid-in Capital Increase Decision,56,62,118


In [10]:
category_df.sum()


category_kr            단일판매ㆍ공급계약체결매출액변동특허권취득자기주식취득 신탁계약 체결 결정타법인 주식 및...
category_en            Single Sales/Supply Contract ExecutionRevenue ...
observations_before                                                 5072
observations_after                                                  2378
observations_total                                                  7450
dtype: object

In [11]:
for m in range(1, 11):
    plus_cols = [
        f"abn_ret_minus_{abs(i)}m" if i < 0 else f"abn_ret_{i}m"
        for i in range(-m+1, m + 1)
    ]
    df_total[f"CAR_{m}m"] = df_total[plus_cols].sum(axis=1)

# 방향 정합성

### 📘 모형식 정의

$$
\text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}}
$$

$$
Pr(hit_{i,t} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해 Delta CAR, post CAR 2개의 지표를 사용

$$
t \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}  
$$

$$
CAR_{post,i,t}
$$

$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- delta CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 과잉반응 감소 / 즉시 반영 후 빠른 수렴 → 정보의 선반영 및 사전 확산이 강화 → 정보 효율성 향상”을 의미한다고 볼 수 있다.
- Post CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 이벤트 발표 이후의 정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다고 해석할 수 있다.


# 중립 포함

In [12]:
import sys
sys.path.append('..')
from hypothesis_test import (
    print_sample_summary_with_neutral,
    logistic_hit_delta_with_neutral_log,
    run_logistic_table
)

# === 전체 파이프라인 (중립 포함) ===
df = df_total.copy()
df["label_sign"] = df["label"]
df_total_sample = print_sample_summary_with_neutral(df, "label_sign")

# ΔCAR 기준 (pre/post window 차이 사용)
windows = list(range(1, 11))
results_delta_with_neutral = run_logistic_table(
    logistic_hit_delta_with_neutral_log, df_total_sample, windows, label="ΔCAR"
)


=== Sample summary (including neutral) ===
Total events:        7,450
Neutrals:            2,087
  - Positive (1):    4,504
  - Negative (-1):   859


--- ΔCAR 기준 결과 ---
 t beta_star    std  t_stat      p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2  neutral_epsilon
 1 0.2135*** 0.0541   3.947 7.928414e-05       1.238     0.664    0.710     4.58   7450 0.0017  0.0013         0.005073
 2 0.2813*** 0.0553   5.090 3.575960e-07       1.325     0.676    0.734     5.84   7450 0.0029  0.0024         0.005647
 3 0.2923*** 0.0545   5.362 8.219400e-08       1.339     0.659    0.722     6.23   7450 0.0031  0.0027         0.006081
 4 0.2756*** 0.0544   5.069 3.994630e-07       1.317     0.659    0.718     5.89   7450 0.0028  0.0024         0.006141
 5 0.3015*** 0.0543   5.554 2.786900e-08       1.352     0.653    0.718     6.48   7450 0.0033  0.0029         0.006142
 6 0.3369*** 0.0544   6.198 5.710000e-10       1.401     0.648    0.721     7.25   7450 0.0042  0.0037        

# 중립 제거

In [13]:
import sys
sys.path.append('..')
from hypothesis_test import (
    print_sample_summary,
    logistic_hit_delta_log,
    run_logistic_table
)

# === 전체 파이프라인 ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_nn = print_sample_summary(df, "label_sign")

# ΔCAR 기준 (pre/post window 차이 사용)
windows = list(range(1, 11))
results_delta = run_logistic_table(logistic_hit_delta_log, df_nn, windows, label="ΔCAR", include_r2=False)


=== Sample summary (neutral removed) ===
Total events:        7,450
Removed neutrals:    2,087
Used (non-neutral):  5,363
  - Positive (1):    4,504
  - Negative (-1):   859


--- ΔCAR 기준 결과 ---
 t  beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs
 1 -0.3065*** 0.0634  -4.832 0.000001       0.736     0.773    0.715    -5.81   5363
 2 -0.2220*** 0.0649  -3.419 0.000629       0.801     0.782    0.742    -4.02   5363
 3  -0.1730** 0.0636  -2.720 0.006520       0.841     0.763    0.730    -3.27   5363
 4   -0.1287* 0.0631  -2.039 0.041467       0.879     0.753    0.728    -2.47   5363
 5    -0.0672 0.0627  -1.071 0.283947       0.935     0.742    0.729    -1.31   5363
 6    -0.0415 0.0627  -0.662 0.507801       0.959     0.739    0.730    -0.81   5363
 7    -0.0701 0.0619  -1.133 0.257328       0.932     0.731    0.717    -1.40   5363
 8   -0.1278* 0.0612  -2.090 0.036602       0.880     0.726    0.700    -2.61   5363
 9    -0.0937 0.0617  -1.518 0.129033   